<a href="https://colab.research.google.com/github/TummalaSharmila/MachineLearning_in_BI/blob/main/DrWangLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import muon as mu
import scanpy as sc
import celloracle as co
import pandas as pd
import numpy as np

# Load MuData file
mdata = mu.read("your_file.h5mu")

# Extract RNA modality (this is an AnnData object)
adata = mdata.mod['rna']

# Option 1: use 'Day' as proxy for cell type
adata.obs["cell_type"] = adata.obs["Day"].astype(str)

# Option 2: assign dummy label
# adata.obs["cell_type"] = "unspecified"

#preprocessing
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
sc.pp.scale(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

#create cell oracle Object
oracle = co.Oracle()
oracle.import_anndata_as_raw_data(adata)
oracle.cell_type_column_name = "cell_type"

#gene annotatins and GRN buidling
oracle.get_gene_annotation(genome="mm10")  # Or "hg38" for human

oracle.build_grn_for_simulation(
    input_tf_names="auto",
    verbose=True
)

#dimensional reduction
oracle.perform_pca()
oracle.reduce_dimension(method="umap")
oracle.calculate_velocity_on_grid()

# Visualize
oracle.plot_vector_field()

#save files

# Create UMAP + vector field plot
fig = oracle.plot_vector_field(show_arrow=True, color="cell_type", show_legend=True)

# Save to file
fig.savefig("vector_field.png", dpi=300, bbox_inches='tight')



In [ ]:
import numpy as np
import scipy.sparse as sp

# Use raw counts
adata.X = adata.layers["counts"].copy()

# Step 1: Replace NaNs and Infs (for dense or sparse matrices)
if sp.issparse(adata.X):
    adata.X.data[np.isnan(adata.X.data)] = 0
    adata.X.data[np.isinf(adata.X.data)] = 0
else:
    adata.X[np.isnan(adata.X)] = 0
    adata.X[np.isinf(adata.X)] = 0

# Step 2: Remove cells and genes with all-zero expression
sc.pp.filter_cells(adata, min_counts=1)
sc.pp.filter_genes(adata, min_counts=1)

# Step 3: Continue with standard processing
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
sc.pp.scale(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)


In [ ]:
# Create CellOracle object
oracle = co.Oracle()

# Correct method for v0.20.0
oracle.set_data(adata, cell_type_column_name="cell_type")



In [ ]:
# --------------------------
# 1. Imports and Data Setup
# --------------------------
import muon as mu
import scanpy as sc
import celloracle as co
import numpy as np
import pandas as pd
import os

# Load your MuData
mdata = mu.read("your_file.h5mu")

# Extract RNA modality as AnnData
adata = mdata.mod["rna"]

# Use 'Day' as a cell type label (CellOracle needs this)
adata.obs["cell_type"] = adata.obs["Day"].astype(str)

# --------------------------
# 2. Preprocessing
# --------------------------

# Replace .X with raw counts
adata.X = adata.layers["counts"].copy()

# Fix any NaN or Inf issues in sparse or dense matrix
import scipy.sparse as sp
if sp.issparse(adata.X):
    adata.X.data[np.isnan(adata.X.data)] = 0
    adata.X.data[np.isinf(adata.X.data)] = 0
else:
    adata.X[np.isnan(adata.X)] = 0
    adata.X[np.isinf(adata.X)] = 0

# Filter zero-expression cells and genes
sc.pp.filter_cells(adata, min_counts=1)
sc.pp.filter_genes(adata, min_counts=1)

# Normalize + log1p
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Select HVGs
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)

# Scale and reduce
sc.pp.scale(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

# --------------------------
# 3. CellOracle Setup
# --------------------------

# Create oracle object
oracle = co.Oracle()
oracle.import_anndata_as_raw_count(adata)  # method for your version
oracle.cell_type_column_name = "cell_type"

# --------------------------
# 4. Gene Regulatory Network
# --------------------------

# Pull gene info from mm10
oracle.get_gene_annotation(genome="mm10")

# Build GRN
oracle.build_grn_for_simulation(input_tf_names="auto", verbose=True)

# --------------------------
# 5. Dimensionality Reduction for Simulations
# --------------------------

oracle.perform_PCA()
oracle.embedding_name = "umap"
oracle.calculate_grid_arrows()

# --------------------------
# 6. Visualization
# --------------------------

# Plot vector field
fig = oracle.plot_simulation_flow_on_grid(color="cell_type", show_arrow=True)
fig.savefig("figures/vector_field.png", dpi=300, bbox_inches="tight")

# Plot expression of a key gene (e.g., Gata1)
fig = oracle.plot_quiver("Gata1", color_map="coolwarm")
fig.savefig("figures/Gata1_expression_quiver.png", dpi=300, bbox_inches="tight")


In [ ]:
# After running UMAP
sc.tl.umap(adata)

# Rename UMAP embedding to what CellOracle expects
adata.obsm["umap"] = adata.obsm["X_umap"]

# Create Oracle and import data
oracle = co.Oracle()
oracle.import_anndata_as_normalized_count(adata)
oracle.cell_type_column_name = "cell_type"


In [ ]:
# Run UMAP
sc.tl.umap(adata)

# Create CellOracle object
oracle = co.Oracle()

# Explicitly tell CellOracle where to find the embedding and clusters
oracle.import_anndata_as_normalized_count(
    adata,
    embedding_name="X_umap",
    cluster_column_name="cell_type"
)
oracle.cell_type_column_name = "cell_type"


In [ ]:
# Run UMAP
sc.tl.umap(adata)

# Set the required 'raw_count' layer
adata.layers["raw_count"] = adata.layers["counts"].copy()

# Create Oracle and import
oracle = co.Oracle()
oracle.import_anndata_as_normalized_count(
    adata,
    embedding_name="X_umap",
    cluster_column_name="cell_type"
)
oracle.cell_type_column_name = "cell_type"


In [ ]:
# Add this BEFORE calling import
adata.layers["raw_count"] = adata.layers["counts"].copy()

# Create Oracle and call the method with embedding_name and cluster_column_name
oracle = co.Oracle()
oracle.import_anndata_as_normalized_count(
    adata,
    embedding_name="X_umap",
    cluster_column_name="cell_type"
)
oracle.cell_type_column_name = "cell_type"


In [ ]:
# ------------------------------------------
# 1. Imports and Setup
# ------------------------------------------
import muon as mu
import scanpy as sc
import celloracle as co
import numpy as np
import pandas as pd
import os
import scipy.sparse as sp

# Create output folder
os.makedirs("figures", exist_ok=True)

# ------------------------------------------
# 2. Load your MuData and extract RNA
# ------------------------------------------
mdata = mu.read("your_file.h5mu")  # replace with actual filename
adata = mdata.mod["rna"]

# Use 'Day' as a proxy for cell type
adata.obs["cell_type"] = adata.obs["Day"].astype(str)

# ------------------------------------------
# 3. Preprocess: log-normalized UNSCALED data
# ------------------------------------------

# Start from raw counts
adata.X = adata.layers["counts"].copy()

# Clean up NaNs and Infs
if sp.issparse(adata.X):
    adata.X.data[np.isnan(adata.X.data)] = 0
    adata.X.data[np.isinf(adata.X.data)] = 0
else:
    adata.X[np.isnan(adata.X)] = 0
    adata.X[np.isinf(adata.X)] = 0

# Filter low-quality cells and genes
sc.pp.filter_cells(adata, min_counts=1)
sc.pp.filter_genes(adata, min_counts=1)

# Normalize and log1p
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Run HVG, PCA, neighbors, and UMAP
sc.pp.highly_variable_genes(adata, n_top_genes=2000, subset=True)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

# Set required 'raw_count' layer for CellOracle
adata.layers["raw_count"] = adata.layers["counts"].copy()

# ------------------------------------------
# 4. Create CellOracle Object and Import Data
# ------------------------------------------
oracle = co.Oracle()

oracle.import_anndata_as_normalized_count(
    adata,
    embedding_name="X_umap",           # required for vector field
    cluster_column_name="cell_type"    # used for coloring
)

oracle.cell_type_column_name = "cell_type"

# ------------------------------------------
# 5. Gene Regulatory Network (GRN) Inference
# ------------------------------------------
oracle.get_gene_annotation(genome="mm10")

oracle.build_grn_for_simulation(
    input_tf_names="auto",  # optional: can also pass custom TF list
    verbose=True
)

# ------------------------------------------
# 6. Dimensionality Reduction for Oracle
# ------------------------------------------
oracle.perform_PCA()
oracle.embedding_name = "umap"  # use UMAP space
oracle.calculate_grid_arrows()

# ------------------------------------------
# 7. Visualizations & Save Outputs
# ------------------------------------------

# Plot vector field (cell movement)
fig = oracle.plot_simulation_flow_on_grid(color="cell_type", show_arrow=True)
fig.savefig("figures/vector_field.png", dpi=300, bbox_inches="tight")

# Plot expression + arrows for a specific gene (e.g. Gata1)
fig = oracle.plot_quiver("Gata1", color_map="coolwarm")
fig.savefig("figures/Gata1_expression_quiver.png", dpi=300, bbox_inches="tight")


In [ ]:
# ---------------------------------------------
# 0. Imports and Environment Setup
# ---------------------------------------------
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import mudata
import matplotlib.pyplot as plt
import celloracle as co

plt.rcParams["figure.figsize"] = [6, 4.5]
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Create folder to save output
save_folder = "figures"
os.makedirs(save_folder, exist_ok=True)

# ---------------------------------------------
# 1. Load and Prepare MuData
# ---------------------------------------------
file_path = "your_file.h5mu"  # <<< CHANGE THIS TO YOUR ACTUAL FILE
mdata = mudata.read_h5mu(file_path)

# Extract RNA & ATAC modalities
adata_rna = mdata["rna"]
adata_atac = mdata["atac"]

# Create AnnData object from RNA data
adata = anndata.AnnData(
    X=adata_rna.X.copy(),
    obs=adata_rna.obs.copy(),
    var=adata_rna.var.copy()
)

# Add raw counts and a cell type label
adata.layers["raw_count"] = adata_rna.layers["counts"].copy()
adata.obs["cell_type"] = adata.obs["Day"].astype(str)

# ---------------------------------------------
# 2. Preprocessing (Filtering, Normalization, UMAP)
# ---------------------------------------------
# Clean up
adata.X = adata.layers["raw_count"].copy()

# Filter
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.filter_genes(adata, min_counts=10)

# Normalize & log1p (no scaling!)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000, subset=True)

# Dimensionality reduction
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

# Optional: downsample to 30K cells
if adata.shape[0] > 30000:
    sc.pp.subsample(adata, n_obs=30000, random_state=123)

print(f"Final shape: Cells={adata.shape[0]}, Genes={adata.shape[1]}")

# ---------------------------------------------
# 3. Load Prebuilt Base GRN
# ---------------------------------------------
base_GRN = co.data.load_mouse_scATAC_atlas_base_GRN()
base_GRN.head()

# ---------------------------------------------
# 4. Create Oracle and Load Data
# ---------------------------------------------
oracle = co.Oracle()

# Ensure raw counts are used
adata.X = adata.layers["raw_count"].copy()

# Load into Oracle
oracle.import_anndata_as_raw_count(
    adata=adata,
    cluster_column_name="cell_type",
    embedding_name="X_umap"
)

# Load prebuilt TF-target data
oracle.import_TF_data(TF_info_matrix=base_GRN)

# ---------------------------------------------
# 5. KNN Imputation
# ---------------------------------------------
# PCA
oracle.perform_PCA()

# Automatically determine #PCs
plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_))
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_)) > 0.002))[0][0]
n_comps = min(n_comps, 50)
print(f"Using {n_comps} PCs")

# Run KNN imputation
n_cell = oracle.adata.shape[0]
k = int(0.025 * n_cell)
oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8, b_maxl=k*4, n_jobs=4)

# Save Oracle object
oracle.to_hdf5("oracle_model.celloracle.oracle")

# ---------------------------------------------
# 6. GRN Construction (per cluster)
# ---------------------------------------------
links = oracle.get_links(
    cluster_name_for_GRN_unit="cell_type",
    alpha=10,
    verbose_level=10
)

# Save Links object
links.to_hdf5("links.celloracle.links")

# ---------------------------------------------
# 7. Filter & Analyze GRNs
# ---------------------------------------------
# Filter edges by significance and weight
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)

# Degree distribution plot
plt.rcParams["figure.figsize"] = [9, 4.5]
links.plot_degree_distributions()

# Compute network scores
links.get_network_score()
print(links.merged_score.head())

# Save filtered GRNs
links.to_hdf5("links.filtered.celloracle.links")
